In [28]:
from preprocessor import *

In [29]:
data = pd.read_csv('~/Documents/datasets/house_prices/train.csv')

prep = Preprocessor()

data = data.drop(['Id'],axis=1)
categorical, numerical = prep.column_types(data)

data[categorical] = data[categorical].fillna("None",axis = 1)
data[numerical] = data[numerical].fillna(0,axis = 1)
print(data.shape)

(1460, 80)


In [30]:
print(data.shape)
data.head()

(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [31]:
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.svm import SVR

sc = MinMaxScaler()

X_cats = pd.get_dummies(data[categorical])
X_cats_df = pd.DataFrame(X_cats)

X_nums = sc.fit_transform(data[numerical])
X_nums_df = pd.DataFrame(X_nums,columns = numerical)

X_all = pd.concat([X_nums_df,X_cats_df])
X_all.fillna(0,axis = 1,inplace =True)

X = X_all.drop(['SalePrice'],axis=1)
y = X_all['SalePrice']




print("X shape: {}".format(X_all.shape))
print("Labels shape: {}".format(y.shape))


X_train, X_test, y_train ,y_test = train_test_split(X, y,test_size=0.25, random_state= 42,shuffle=True)

X shape: (2920, 304)
Labels shape: (2920,)


In [32]:
X_all.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley_Grvl,Alley_None,Alley_Pave,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,...,Street_Grvl,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,Utilities_AllPub,Utilities_NoSeWa,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,0.119780,0.413559,0.0,0.0,0.0,0.0,0.375,0.0,0.0,0.0,...,0.0,0.0,0.500000,0.140098,0.0,0.0,0.000000,0.949275,0.883333,0.50
1,0.212942,0.000000,0.0,0.0,0.0,0.0,0.375,0.0,0.0,0.0,...,0.0,0.0,0.333333,0.206547,0.0,0.0,0.347725,0.753623,0.433333,0.25
2,0.134465,0.419370,0.0,0.0,0.0,0.0,0.375,0.0,0.0,0.0,...,0.0,0.0,0.333333,0.150573,0.0,0.0,0.000000,0.934783,0.866667,0.50
3,0.143873,0.366102,0.0,0.0,0.0,0.0,0.375,0.0,0.0,0.0,...,0.0,0.0,0.416667,0.123732,0.0,0.0,0.000000,0.311594,0.333333,0.00
4,0.186095,0.509927,0.0,0.0,0.0,0.0,0.500,0.0,0.0,0.0,...,0.0,0.0,0.583333,0.187398,0.0,0.0,0.224037,0.927536,0.833333,0.50


In [33]:
clf = RandomForestRegressor()
clf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [34]:
print('Accuracy on the training subset: {:.3f}'.format(clf.score(X_train, y_train)))
print('Accuracy on the testing subset: {:.3f}'.format(clf.score(X_test, y_test)))

Accuracy on the training subset: 0.988
Accuracy on the testing subset: 0.953


In [35]:
filename = 'model.pkl'
joblib.dump(clf,filename)

['model.pkl']